# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Doc2Vec 
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import doc2vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
import dask.dataframe as dd

In [4]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [5]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## TaggedDocument For Doc2Vec

In [6]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Load Data

#### Path

In [13]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    news_senti_outcome = '/Volumes/disk1/outcome_for_News_sentiment_analysis/'
    daumCommentsPath = '/Volumes/disk1/data/daum_Comments/'
    naverCommentsPath = '/Volumes/disk1/data/naver_Comments/'
    outcomeDaumCommentsPath = '/Volumes/disk1/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = '/Volumes/disk1/outcome_comments_for_naver/'
    outcome_predata = 'Volumes/disk1/pre_data_for_comments/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'
    news_senti_outcome = './outcome_for_News_sentiment_analysis/'
    daumCommentsPath = 'd:/data/daum_Comments/'
    naverCommentsPath = 'd:/data/naver_Comments/'
    outcomeDaumCommentsPath = 'd:/outcome_comments_for_daum/'
    outcomeNaverCommentsPath = 'd:/outcome_comments_for_naver/'
    outcome_predata = 'd:/pre_data_for_comments/'

#### News 

In [8]:
os.listdir(news_senti_outcome)

['daum_news_sentiment_analysis.csv', 'naver_news_sentiment_analysis.csv']

In [9]:
# Naver
naverData = pd.read_csv(os.path.join(news_senti_outcome, 'naver_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
naverData['site'] = ['Naver'] * naverData.shape[0]
reNaverData = naverData[naverData.number_of_crawled_comment != 0]
print (reNaverData.shape)
reNaverData.head()

(15015, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a29c445588c132954d1973a,정치,2017.12.07,연합뉴스,1713,1465,1,"北외무성 ""전쟁 바라지 않지만 결코 피하지 않을 것""","美고위인사 대북언급 비난하며 ""전쟁 기정사실화"" 위협 며칠 새 이어지는 북한 군민...","['외무성', '핵전쟁', '대변인']","{'대변인', '조선반도', '북한', '미국', '중앙', '도화선', '핵전쟁'...",86.0,22.0,negative,Naver
5a29c445588c132954d1973b,정치,2017.12.07,한국일보,2551,2062,2,"예산전쟁, 예결위 간사ㆍ호남이 웃었다",예결위 간사들이 최대 수혜자..당 지도부 내 몫 챙기기도 여전 황주홍ㆍ김도읍 등...,"['예산', '예결위', 'soc']","{'정부안', '호남', '예산안', '지역구', '의원', '증액', '국민의당'}",46.0,62.0,positive,Naver
5a29c445588c132954d1973c,정치,2017.12.07,뉴시스,610,536,3,"혐의 부인에 20시간 조사…檢, 최경환 구속 카드 꺼내나",【서울=뉴시스】 최진석 기자 = 박근혜 정부 시절 국가정보원 특수활동비 수수 의혹 ...,"['최경환', '구속영장', '국가정보원']","{'국정원장', '검찰', '조사', '혐의', '정기국회', '의원', '구속영장...",77.0,31.0,negative,Naver
5a29c445588c132954d1973d,정치,2017.12.07,연합뉴스,145,133,4,"최재형 감사원장 후보자 ""독립성 강화는 임명권자의 뜻""",감사원장에 내정된 최재형 사법연수원장(고양=연합뉴스) 이희열 기자 = 7일 감사원장...,"['이슈 · 최재형 감사원장 내정', '감사원장', '최재형', '감사원']","{'생활', '법관', '지명', '감사원장', '공직 사회', '후보자'}",39.0,69.0,positive,Naver
5a29c445588c132954d1973e,정치,2017.12.07,동아일보,1074,932,5,"B-1B 한반도에 뜨자, 평양 비운 김정은",[동아일보] 북중 접경지 양강도 삼지연 시찰… 방북 유엔 사무차장 면담 안할듯 B-...,"['김정은', 'b-1b', '한반도']","{'김정은', '훈련', '삼지연', '접경', '사무차장', '공장', '펠트먼'...",72.0,36.0,negative,Naver


In [10]:
# Daum
daumData = pd.read_csv(os.path.join(news_senti_outcome, 'daum_news_sentiment_analysis.csv'), index_col=0, header= 0, encoding = 'utf-8')
daumData['site'] = ['daum'] * daumData.shape[0]
reDaumData = daumData[daumData.number_of_crawled_comment != 0]
print (reDaumData.shape)
reDaumData.head()

(9369, 14)


,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords,negative,positive,Decision,site
id,,,,,,,,,,,,,,
5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,903,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","['부작용', '비아그라', '아일랜드']","{'연기', '부작용', '남성들', '지역', '건강', '공장', '세보 효과'}",50.0,58.0,positive,daum
5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"['커피', '낮잠', '음식물']","{'입냄새', '커피', '디스크', '점심 식사', '낮잠', '식후', '치아'...",81.0,27.0,negative,daum
5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,807,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"['살인혐의', '철원', '검찰송치']","{'남편', '경찰', '철원', '아들', '진술', '범행', '주변', '앙심...",99.0,9.0,negative,daum
5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,368,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\r\n신영자, 얼마를 어떻게 받았나 [...","['신영자', '갑질', '롯데백화점']","{'검찰', '신영자 이사장', '매장', '혐의', '유통업체', '네이처리퍼블릭...",77.0,31.0,negative,daum
5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,367,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"['공범', '살인', '과자']","{'경찰', '유치장', '폭행', '쪽지', '과자', '혐의', '범행', '남...",102.0,6.0,negative,daum


### 댓글 파일

In [17]:
os.listdir(outcome_predata)

['predata_for_daum_news_comment.csv', 'predata_for_naver_news_comment.csv']

In [21]:
dfNaver = pd.read_csv(outcome_predata +  'predata_for_naver_news_comment.csv', header = 0, encoding = 'utf-8')
dfNaver.shape

(11889935, 9)

In [24]:
dfDaum = pd.read_csv(outcome_predata +  'predata_for_daum_news_comment.csv', header = 0, encoding = 'utf-8')
dfDaum.shape

(4924599, 9)

(4924599, 8)
Wall time: 2h 20min 13s


### Twitter & Mecab
> * 메모리, 시간상의 문제점
>> * 메모리는 dataframe apply로 하면 kernel이 죽어버림.
>> * 시간 for 문으로 할 수 있는건 한번에 해버리는게 낫겠지? 

In [13]:
taggerName_ct = 'ct'
print ('{}'.format(taggerName_ct))
model1_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-ct.model')
name1_ct = '-'.join(re.split('[\(\),\/]',str(model1_ct)))+taggerName_ct
print ('Load model')
model2_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-ct.model')
name2_ct = '-'.join(re.split('[\(\),\/]',str(model2_ct)))+taggerName_ct
print ('Load model')
model3_ct = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-ct.model')
name3_ct = '-'.join(re.split('[\(\),\/]',str(model3_ct)))+taggerName_ct
print ('Load model')
taggerName_mecab = 'mecab'
print ('{}'.format(taggerName_mecab))
model1_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-10_negative-7_hs-0_dm-1_dm_concat-0_dm_mean-1_by-mecab.model')
name1_mecab = '-'.join(re.split('[\(\),\/]',str(model1_mecab)))+taggerName_mecab
print ('Load model')
model2_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-5_negative-7_hs-0_dm-1_dm_concat-1_dm_mean-0_by-mecab.model')
name2_mecab = '-'.join(re.split('[\(\),\/]',str(model2_mecab)))+taggerName_mecab
print ('Load model')
model3_mecab = doc2vec.Doc2Vec.load(loadModelPath+'doc2vec_size-1000_epoch-20_window-None_negative-7_hs-0_dm-0_dm_concat-0_dm_mean-0_by-mecab.model')
name3_mecab = '-'.join(re.split('[\(\),\/]',str(model3_mecab)))+taggerName_mecab
print ('Load model')

ct
Load model
Load model
Load model
mecab
Load model
Load model
Load model


In [14]:
%%time
for idx in reNaverData.index:
    commentFile = os.path.join(naverCommentsPath, idx+'.csv')
    '''
    if os.path.isfile(commentFile):
        print (idx)
        data = reNaverData.loc[idx]
        ## model 1 ct
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, ct, stopwords, TaggedDocument, model1_ct, name1_ct)
        ## model 2 ct
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, ct, stopwords, TaggedDocument, model2_ct, name2_ct)
        ## model 3 ct
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, ct, stopwords, TaggedDocument, model3_ct, name3_ct)  

        ## model 1 mecab
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, mecab, stopwords, TaggedDocument, model1_mecab, name1_mecab)
        ## model 2 mecab
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, mecab, stopwords, TaggedDocument, model2_mecab, name2_mecab)
        ## model 3 mecab
        bm.PipeLineForSentimentAnalysis(naverCommentsPath, classifierPath, outcomeNaverCommentsPath, data, mecab, stopwords, TaggedDocument, model3_mecab, name3_mecab)  
    '''

UnicodeDecodeError: 'cp949' codec can't decode byte 0x90 in position 52: illegal multibyte sequence

In [15]:
%%time
for idx in reDaumData.index:
    commentFile = os.path.join(daumCommentsPath, idx+'.csv')
    if os.path.isfile(commentFile):
        if not os.path.isdir(os.path.join(daumCommentsPath, idx)):
            data = reDaumData.loc[idx]
            ## model 1 ct
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, ct, stopwords, TaggedDocument, model1_ct, name1_ct)
            ## model 2 ct
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, ct, stopwords, TaggedDocument, model2_ct, name2_ct)
            ## model 3 ct
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, ct, stopwords, TaggedDocument, model3_ct, name3_ct)  

            ## model 1 mecab
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, mecab, stopwords, TaggedDocument, model1_mecab, name1_mecab)
            ## model 2 mecab
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, mecab, stopwords, TaggedDocument, model2_mecab, name2_mecab)
            ## model 3 mecab
            bm.PipeLineForSentimentAnalysis(daumCommentsPath, classifierPath, outcomeDaumCommentsPath, data, mecab, stopwords, TaggedDocument, model3_mecab, name3_mecab)  

UnicodeDecodeError: 'cp949' codec can't decode byte 0x90 in position 52: illegal multibyte sequence